In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
AssistantAgent?

Init signature:
AssistantAgent(
    name: str,
    model_client: autogen_core.models._model_client.ChatCompletionClient,
    *,
    tools: Optional[List[Union[autogen_core.tools._base.BaseTool[Any, Any], Callable[..., Any], Callable[..., Awaitable[Any]]]]] = None,
    handoffs: Optional[List[autogen_agentchat.base._handoff.Handoff | str]] = None,
    model_context: autogen_core.model_context._chat_completion_context.ChatCompletionContext | None = None,
    description: str = 'An agent that provides assistance with ability to use tools.',
    system_message: str | None = 'You are a helpful AI assistant. Solve tasks using your tools. Reply with TERMINATE when the task has been completed.',
    model_client_stream: bool = False,
    reflect_on_tool_use: bool = False,
    tool_call_summary_format: str = '{result}',
    memory: Optional[Sequence[autogen_core.memory._base_memory.Memory]] = None,
)
Docstring:     
An agent that provides assistance with tool use.

The :meth:`on_messages` retur

In [2]:
import asyncio
import httpx
import logging
import json
import sys
import os
from pathlib import Path
import time
#wechat search and send

In [102]:
processed_params= {"contact_name": "陈浩",
                    "message": "测试微信数据接口 "
}
async with httpx.AsyncClient() as client:
    tool_response= await client.post(
    "http://localhost:8003/tools/wechat/search_and_send",
    json=processed_params,
    timeout=300.0
    )
tool_response.raise_for_status()
print(tool_response.json())

{'status': 'success', 'message': '成功向联系人 陈浩 发送消息', 'contact_name': '陈浩', 'contact_id': ''}


In [25]:
# Define a tool that searches the web for information.
# async def citicbank_card_info(name:str) -> str:
#     """Find information on the citicbank web"""
#     message=f'查询{name}的帐单。'
#     return message
# async def wechat_send(name:str,message: str) -> str:
#     """Send information by wechat"""
#     processed_params= {"contact_name": name,
#                     "message": message}
#     async with httpx.AsyncClient() as client:
#         tool_response= await client.post(
#         "http://localhost:8003/tools/wechat/search_and_send",
#         json=processed_params,
#         timeout=300.0
#         )
#     return tool_response.json()

async def wechat_send(name:str,message: str) -> str:
    """Send information by wechat"""

    return f'send {message} to name !'

# Create an agent that uses the OpenAI GPT-4o model.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     # api_key="YOUR_API_KEY",
# )
model_client = OpenAIChatCompletionClient(
    model="gemma3:27b-tools",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[wechat_send],
    system_message="当对话中提到发送信息时,使用工具完成任务，微信联系人提取对话文中的人名.",
)

In [20]:
# await Console(agent.run_stream(task='写个冒泡排序使用工具微信发给赵子豪'))

In [27]:
# await Console(agent.run_stream(task='写一个agnet工具的创业计划，字数不少于300字，完成后使用微信发给联系人：陈浩'))

In [91]:
# async def assistant_run() -> None:
#     response = await agent.on_messages(
#         [TextMessage(content="写一个冒泡排序的代码。写完全部代码后再发陈浩。", source="user")],
#         cancellation_token=CancellationToken(),
#     )
#     print(response.inner_messages)
#     print(response.chat_message)


# # Use asyncio.run(assistant_run()) when running in a script.
# await assistant_run()

In [ ]:
await Console(team.run_stream(task=task))

In [ ]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        WechatAgent: send message by wechat

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[citicbank_card_info],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

wechat_send_agent = AssistantAgent(
    "WechatAgent",
    description="An agent for send message by wechat.",
    model_client=model_client,
    tools=[wechat_send],
    system_message="""
    You are a message send tool.
    Given the tasks you have been assigned, you should extract the name and message when use this tool
    """,
)

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [ ]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [ ]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [ ]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [ ]:
await Console(team.run_stream(task=task))

In [ ]:
import base64

def is_image_path(text):
    image_extensions = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".tif")
    if text.endswith(image_extensions):
        return True
    else:
        return False

def encode_image(image_path):
    """Encode image file to base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")